In [97]:
from pathlib import Path
from decode_fish.funcs.utils import *
import glob

base_string = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   

def make_sweep_str(pfile, pars, group):
    
    base_str = base_string + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        base_str += ' ' + k + f'={pars[k]}'
        name += k.split('.')[-1]+f':{pars[k]}x'
    name = name[:-1]
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(pfile, name, group):
    
    base_str = base_string + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [98]:
# group = 'sweep_4'
# pfile = 'covid_early_smFISH_6'

group = 'sweep_start'
pfile = 'msp300_n3'

variable_col = param_iter()
variable_col.add('training.start_ae', 0, 1000, 2000, 3000, 5000)


par_prod = variable_col.param_product()

# name = 'Ref_SI'

In [99]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group)
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=msp300_n3 training.start_ae=0 run_name=start_ae:0 output.group=sweep_start

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=msp300_n3 training.start_ae=1000 run_name=start_ae:1000 output.group=sweep_start

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=msp300_n3 training.start_ae=2000 run_name=start_ae:2000 output.group=sweep_start

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log

In [103]:
exps = ['msp300', 'covid_early', 'N2_352', 'rab11', 'scrib', 'sgg', 'Dlg1']
ver = 'b1'
name = 'b2'

with open('run_slurm.sh', 'w') as f:
    for exp in exps:  
        exec_str = base_string + f' +experiment={exp}_{ver} run_name={name}' + '\n'
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=msp300_b1

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=covid_early_b1

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=N2_352_b1

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=rab11_b1

